In [6]:
import numpy as np
import h5py
import argparse
import sys
import re

In [27]:
# Different data sets to try.
# Note: TREC has no development set.
# And SUBJ and MPQA have no splits (must use cross-validation)
FILE_PATHS = {"SST1": ("data/stsa.fine.phrases.train",
                       "data/stsa.fine.dev",
                       "data/stsa.fine.test"),
              "SST2": ("data/stsa.binary.phrases.train",
                       "data/stsa.binary.dev",
                       "data/stsa.binary.test"),
              "TREC": ("data/TREC.train.all", None,
                       "data/TREC.test.all"),
              "SUBJ": ("data/subj.all", None, None),
              "MPQA": ("data/mpqa.all", None, None)}

# Kaggle dataset for now
args = {"SST1"}

In [35]:
def line_to_words(line, dataset):
    # Different preprocessing is used for these datasets.
    if dataset not in ['SST1', 'SST2']:
        clean_line = clean_str_sst(line.strip())
    else:
        clean_line = clean_str(line.strip())
    clean_line = clean_str_sst(line.strip())
    words = clean_line.split(' ')
    words = words[1:]
    return words

In [34]:
def get_vocab(file_list, dataset=''):
    """
    Construct index feature dictionary.
    EXTENSION: Change to allow for other word features, or bigrams.
    """
    max_sent_len = 0
    word_to_idx = {}
    # Start at 2 (1 is padding)
    idx = 2
    for filename in file_list:
        if filename:
            with open(filename, "r") as f:
                for line in f:
                    words = line_to_words(line, dataset)
                    max_sent_len = max(max_sent_len, len(words))
                    for word in words:
                        if word not in word_to_idx:
                            word_to_idx[word] = idx
                            idx += 1
    return max_sent_len, word_to_idx

In [32]:
def main(arguments):
    global args
    parser = argparse.ArgumentParser(
        description=__doc__,
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument('dataset', help="Data set",
                        type=str)
    args = parser.parse_args(arguments)
    dataset = args.dataset
    train, valid, test = FILE_PATHS[dataset]
    
    # Features are just the words.
    max_sent_len, word_to_idx = get_vocab([train, valid, test])

In [33]:
main(args)

TypeError: 'Namespace' object is not iterable